In [ ]:
pip install surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from surprise import Reader, Dataset, SVD
from surprise.model_selection import KFold
from surprise.model_selection.validation import cross_validate
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df_meta = pd.read_csv('/content/drive/MyDrive/Datasets/movies_metadata.csv')
df_ratings = pd.read_csv('/content/drive/MyDrive/Datasets/ratings.csv')
df_links = pd.read_csv('/content/drive/MyDrive/Datasets/links.csv')
df_keywords = pd.read_csv('/content/drive/MyDrive/Datasets/keywords.csv')
df_credits = pd.read_csv('/content/drive/MyDrive/Datasets/credits.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195

In [ ]:
df_meta['overview'] = df_meta['overview'].fillna('')

In [ ]:
col=np.array(df_links['tmdbId'], np.int64)
df_links['tmdbId']=col

In [ ]:
df_meta.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,30-10-1995,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,15-12-1995,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,FALSE,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,22-12-1995,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,FALSE,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,22-12-1995,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,FALSE,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,10-02-1995,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [ ]:
df_meta['id'] = df_meta['id'].str.replace('-', '').astype(float)
df_meta['id'] = pd.to_numeric(df_meta['id'])

In [ ]:
df_meta.rename(columns={'id':'tmdbId'}, inplace=True)
df_meta = pd.merge(df_meta,df_links,on='tmdbId')
df_meta.drop(['imdb_id'], axis=1, inplace=True)
df_meta.head(2)

,adult,belongs_to_collection,budget,genres,homepage,tmdbId,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count,movieId,imdbId
0,FALSE,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862.0,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,...,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0,1,114709
1,FALSE,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844.0,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,...,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,2,113497


In [ ]:
tf_idf = TfidfVectorizer(stop_words='english')

tf1=time.perf_counter()
tfidf_matrix = tf_idf.fit_transform(df_meta['overview'])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
tf2=time.perf_counter()
indices = pd.Series(df_meta.index, index=df_meta['original_title']).drop_duplicates()

In [ ]:
reader = Reader()
df = Dataset.load_from_df(df_ratings[['userId', 'movieId', 'rating']], reader)
kf = KFold(n_splits=5)
kf.split(df)

<generator object KFold.split at 0x7f5476ed74d0>

In [ ]:
svd = SVD()

svt1=time.perf_counter()
cross_validate(svd, df, measures=['RMSE', 'MAE'])
svt2=time.perf_counter()

trainset = df.build_full_trainset()

In [ ]:
df_links = df_links.merge(df_meta[['title', 'tmdbId']], on='tmdbId').set_index('title')
links_idx = df_links.set_index('tmdbId')

In [ ]:
def hybrid(userId, title):
    idx = indices[title]
    tmdbId = df_links.loc[title]['tmdbId'] # Get the corresponding tmdb id
    
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31] # Scores of the 30 most similar movies
    movie_indices = [i[0] for i in sim_scores]
    
    movies = df_meta.iloc[movie_indices][['title', 'vote_average', 'tmdbId']]
    movies['est'] = movies['tmdbId'].apply(lambda x: svd.predict(userId, links_idx.loc[x]['movieId']).est) # Estimated prediction using svd
    movies = movies.sort_values('est', ascending=False) # Rank movies according to the predicted values
    movies.columns = ['Title', 'Vote Average', 'TMDb Id', 'Estimated Prediction']
    return movies.head(5)

In [ ]:
hybrid(1, 'Safe Haven')

,Title,Vote Average,TMDb Id,Estimated Prediction
2252,Live and Let Die,6.4,253.0,4.591771
1530,Out of the Past,7.6,678.0,4.550101
2218,Dr. No,6.9,646.0,4.354233
2217,From Russia with Love,6.9,657.0,4.343170
6849,Rachel Getting Married,6.1,14976.0,4.268043


In [ ]:
tic=time.perf_counter()
hybrid(1, 'Safe Haven')
toc=time.perf_counter()

In [ ]:
print(f"Time to make prediction is {toc-tic:0.3f} seconds")

Time to make prediction is 0.015 seconds


In [ ]:
print(f"Time taken to train is {((tf2-tf1)+(svt2-svt1)):0.3f} seconds")

Time taken to train is 27.847 seconds
